In [5]:
import umap as umap
import numpy as np
import pandas as pd

In [20]:
human_activity_recognition: pd.DataFrame = pd.read_csv('data/human_activity_recogniton_with_smartphones/train.csv')
trump_insults: pd.DataFrame = pd.read_csv('data/trump_insults/trump_insult_tweets_2014_to_2021.csv')
google_jobs: pd.DataFrame = pd.read_csv('data/google_jobs/job_skills.csv')

In [12]:
human_activity_recognition.shape

(7352, 563)

In [13]:
pd.unique(human_activity_recognition['Activity'])

array(['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [15]:
trump_insults.head()

,Unnamed: 0,date,target,insult,tweet
0,1,2014-10-09,thomas-frieden,fool,"Can you believe this fool, Dr. Thomas Frieden ..."
1,2,2014-10-09,thomas-frieden,DOPE,"Can you believe this fool, Dr. Thomas Frieden ..."
2,3,2015-06-16,politicians,all talk and no action,Big time in U.S. today - MAKE AMERICA GREAT AG...
3,4,2015-06-24,ben-cardin,It's politicians like Cardin that have destroy...,Politician @SenatorCardin didn't like that I s...
4,5,2015-06-24,neil-young,total hypocrite,"For the nonbeliever, here is a photo of @Neily..."


In [17]:
print(trump_insults.shape)
print(len(pd.unique(trump_insults['target'])))
print(len(pd.unique(trump_insults['insult'])))

(10360, 5)
867
6729


In [23]:
print(google_jobs.shape)
print(len(pd.unique(google_jobs['Title'])))
print(len(pd.unique(google_jobs['Category'])))

(1250, 7)
794
23


In [22]:
google_jobs.head()

,Company,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications
0,Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical prog...",BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...
1,Google,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"Shanghai, China",Drive cross-functional activities in the suppl...,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin..."
2,Google,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"New York, NY, United States",Collect and analyze data to draw insight and i...,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...
3,Google,"Developer Advocate, Partner Engineering",Developer Relations,"Mountain View, CA, United States","Work one-on-one with the top Android, iOS, and...",BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,..."
4,Google,"Program Manager, Audio Visual (AV) Deployments",Program Management,"Sunnyvale, CA, United States",Plan requirements with internal customers.\nPr...,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...
